In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd '/content/gdrive/MyDrive/LDS9_K265_TranHoangBach/Week_2/data_day_3'

Mounted at /content/gdrive
/content/gdrive/MyDrive/LDS9_K265_TranHoangBach/Week_2/data_day_3


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [4]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import Row, SQLContext, SparkSession
sc = SparkContext(master="local", appName="New Spark Context")
spark = SparkSession(sc)
sqlcontext = SQLContext(sc)

In [5]:
data = spark.read.csv('flights_small.csv', header=True, inferSchema=True)
data.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

In [6]:
data.count()

10000

In [7]:
data.createOrReplaceTempView('flights_small')

In [8]:
query = 'SELECT * FROM flights_small'
flights = spark.sql(query)
flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 3 rows



In [9]:
flights = flights.withColumn('duration_hrs', flights.air_time/60)
flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|         2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        1.85|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 3 rows



In [10]:
long_flights1 = flights.where(flights.distance > 2000)
long_flights1.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|     duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|              6.0|
|2014|    1| 13|    2156|       -9|     607|      -15|     AS| N597AS|    24|   SEA| BOS|     290|    2496|  21|    56|4.833333333333333|
|2014|    9| 26|     610|       -5|    1523|       65|     US| N127UW|   616|   SEA| PHL|     293|    2378|   6|    10|4.883333333333334|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
only showing top 3 rows



In [11]:
long_flights1.count()

1481

In [19]:
query = "SELECT count(*) FROM flights_small WHERE distance > 2000"
spark.sql(query).show()

+--------+
|count(1)|
+--------+
|    1481|
+--------+



In [20]:
condition = (flights.air_time >= 300) & (flights.air_time <= 600)
time_flights = flights.where(condition)
time_flights.count()

440

In [21]:
query = "SELECT count(*) FROM flights_small WHERE air_time >= 300 AND air_time <= 600"
spark.sql(query).show()

+--------+
|count(1)|
+--------+
|     440|
+--------+



In [22]:
selected1 = flights.select('origin', 'dest', 'carrier')
selected1.show(3)

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| LAX|     VX|
|   SEA| HNL|     AS|
|   SEA| SFO|     VX|
+------+----+-------+
only showing top 3 rows



In [24]:
filterA = selected1.origin == 'SEA'
filterB = selected1.dest == 'PDX'
selected2 = selected1.where(filterA & filterB)
selected2.count()

157

In [25]:
flights = flights.withColumn('avg_speed', flights.distance/(flights.air_time/60))
speed1 = flights.select('origin', 'dest', 'tailnum', 'avg_speed')
speed1.show(3)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
+------+----+-------+------------------+
only showing top 3 rows



In [30]:
speed2 = flights.selectExpr('origin', 'dest', 'tailnum', 'distance/(air_time/60) as avg_speed')
speed2.show(3)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
+------+----+-------+------------------+
only showing top 3 rows



In [31]:
flights.groupBy('origin').agg({'air_time': 'min'}).where(flights.origin == 'PDX').show()

+------+-------------+
|origin|min(air_time)|
+------+-------------+
|   PDX|          100|
+------+-------------+



In [32]:
flights.groupBy('origin').agg({'distance': 'max'}).where(flights.origin == 'SEA').show()

+------+-------------+
|origin|max(distance)|
+------+-------------+
|   SEA|         2724|
+------+-------------+



In [35]:
flights.groupBy().sum('duration_hrs').show()

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



In [37]:
flights.groupBy('tailnum').count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [38]:
flights.groupBy('origin').count().show()

+------+-----+
|origin|count|
+------+-----+
|   SEA| 6754|
|   PDX| 3246|
+------+-----+



In [40]:
flights.groupBy('origin').agg({'air_time': 'mean'}).show()

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



In [42]:
by_month_dest = flights.groupBy('month', 'dest').agg({'dep_delay': 'mean'}).sort('month')
by_month_dest.show()

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|    1| PDX|-5.6923076923076925|
|    1| JFK| 22.428571428571427|
|    1| DCA|               -1.8|
|    1| MKE|                4.0|
|    1| LAS|  9.857142857142858|
|    1| GEG|  8.666666666666666|
|    1| PHL|               -1.0|
|    1| EWR| 13.076923076923077|
|    1| SEA|  4.428571428571429|
|    1| OAK|  8.290322580645162|
|    1| SBA|-0.6666666666666666|
|    1| DFW|  7.696969696969697|
|    1| SLC|             6.5625|
|    1| ABQ|               -3.0|
|    1| KTN|                9.0|
|    1| RDM|             -1.625|
|    1| MCI|                0.5|
|    1| JNU|                2.6|
|    1| ORD| 19.964285714285715|
|    1| MSP|               11.2|
+-----+----+-------------------+
only showing top 20 rows



In [43]:
airports = spark.read.csv('airports.csv', header=True, inferSchema=True)
airports.show(3)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 3 rows



In [48]:
airports.printSchema()

root
 |-- dest: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



In [44]:
airports.count()

1397

In [45]:
airports = airports.withColumnRenamed('faa', 'dest')
airports.show(3)

+----+--------------------+----------+-----------+----+---+---+
|dest|                name|       lat|        lon| alt| tz|dst|
+----+--------------------+----------+-----------+----+---+---+
| 04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
| 06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
| 06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
+----+--------------------+----------+-----------+----+---+---+
only showing top 3 rows



In [47]:
flights_w_detail_airports = flights.join(airports, on='dest', how='left')
flights_w_detail_airports.show(3)

+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------+------------------+------------------+---------+-----------+---+---+---+
|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|air_time|distance|hour|minute|duration_hrs|         avg_speed|              name|      lat|        lon|alt| tz|dst|
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------+------------------+------------------+---------+-----------+---+---+---+
| LAX|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|     132|     954|   6|    58|         2.2| 433.6363636363636|  Los Angeles Intl|33.942536|-118.408075|126| -8|  A|
| HNL|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|     360|    2677|  10|    40|         6.0| 446.1666666666667|     Honolulu Intl|21.318681|